In [34]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/BigData

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/BigData


In [35]:
#import
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE, Isomap
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, Normalizer
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, mean_absolute_error
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
import random

In [36]:
df = pd.read_csv('./results.csv')
#date 형식 변환
df['date'] = [i.replace('-','') for i in df['date']]

df_old =  pd.read_csv('./results.csv')

In [37]:
#all_list : tournament, country, city, home_team, away_team
all_list = ((df['tournament'].unique()).tolist() + (df['country'].unique()).tolist() + (df['city'].unique()).tolist() + (df['home_team'].unique()).tolist() + (df['away_team'].unique()).tolist())

uni_len = '***'.join(all_list)
nat_enc = tf.keras.preprocessing.text.Tokenizer(num_words=len(all_list),
    filters='',
    lower=False,
    split = '***')
nat_enc.fit_on_texts([uni_len])
nat_dic = nat_enc.word_index

In [38]:
#값을 enumerate로 변환
for step, val in enumerate(df['tournament']):
    df['tournament'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['city']):
    df['city'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['country']):
    df['country'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['home_team']):
    df['home_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['away_team']):
    df['away_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['neutral']):
    if df['neutral'].iloc[step] == False:
        df['neutral'].iloc[step] = 0
    if df['neutral'].iloc[step] == True:
        df['neutral'].iloc[step] = 1

In [39]:
#df_new
df_new = df
#len의 역할?
temp_tor = []
for i in df_old['tournament']:
    temp_tor.append(len(i))
temp_away = []
for i in df_old['away_team']:
    temp_away.append(len(i))
temp_home = []
for i in df_old['home_team']:
    temp_home.append(len(i))
df_new['tor_len'] = temp_tor
df_new['home_len'] = temp_home
df_new['away_len'] = temp_away

#df_new : date 형태 변환 ex. 18721130 -> -4.870880
exp = df_new['date'].to_numpy()
scaler = StandardScaler()
exp_2 = scaler.fit_transform(exp.reshape(-1,1))
df_new['date'] = (np.ravel(exp_2)).tolist()
# 결측값 있는 행 제거
df_new = df_new.dropna()

df_label1 = df_new['home_score']
df_label2 = df_new['away_score']
df_new = df_new.drop(['home_score', 'away_score'], axis = 1)


In [40]:
x_train = df_new.to_numpy()

In [41]:
#df_label
df_train = pd.DataFrame(data=x_train, columns=df_new.columns)
df_label = pd.concat([df_label1, df_label2],axis = 1)
df_label.columns = ['home_score', 'away_score']
#win column 추가
win = []
for h, a in zip(df_label['home_score'], df_label['away_score']):
    if h > a:
        win.append('home_win')
    if a > h:
        win.append('away_win')
    if a == h:
        win.append('draw')
df_label['win'] = win


In [42]:
df_label

,home_score,away_score,win
0,0.0,0.0,draw
1,4.0,2.0,home_win
2,2.0,1.0,home_win
3,2.0,2.0,draw
4,3.0,0.0,home_win
...,...,...,...
44099,2.0,1.0,home_win
44100,0.0,5.0,away_win
44101,2.0,0.0,home_win
44102,0.0,0.0,draw


In [43]:
#x_train = df_new.to_numpy()
Norm = Normalizer()
x_temp = Norm.fit_transform(x_train)
df_temp = pd.DataFrame(data=x_temp, columns=df_new.columns)

In [44]:
df_corr = df_temp.corr()

In [45]:
df_label.drop('win', axis=1)

,home_score,away_score
0,0.0,0.0
1,4.0,2.0
2,2.0,1.0
3,2.0,2.0
4,3.0,0.0
...,...,...
44099,2.0,1.0
44100,0.0,5.0
44101,2.0,0.0
44102,0.0,0.0


In [46]:
df_temp.drop('city', axis=1)

,date,home_team,away_team,tournament,country,neutral,tor_len,home_len,away_len
0,-0.008538,0.061351,0.063104,0.534631,0.061351,0.0,0.014023,0.014023,0.012270
1,-0.008459,0.063005,0.061254,0.533789,0.063005,0.0,0.014001,0.012251,0.014001
2,-0.008401,0.061351,0.063104,0.534631,0.061351,0.0,0.014023,0.014023,0.012270
3,-0.008317,0.063005,0.061255,0.533789,0.063005,0.0,0.014001,0.012251,0.014001
4,-0.008259,0.061351,0.063104,0.534632,0.061351,0.0,0.014023,0.014023,0.012270
...,...,...,...,...,...,...,...,...,...
44099,0.000498,0.029217,0.014203,0.123765,0.029217,0.0,0.003246,0.002435,0.003246
44100,0.001077,0.152813,0.036886,0.267862,0.152813,0.0,0.007026,0.017565,0.007904
44101,0.000851,0.135273,0.133885,0.211581,0.135273,0.0,0.005550,0.006937,0.006937
44102,0.001753,0.101466,0.141481,0.451597,0.101466,0.0,0.014291,0.012862,0.010004


In [47]:
df_label

,home_score,away_score,win
0,0.0,0.0,draw
1,4.0,2.0,home_win
2,2.0,1.0,home_win
3,2.0,2.0,draw
4,3.0,0.0,home_win
...,...,...,...
44099,2.0,1.0,home_win
44100,0.0,5.0,away_win
44101,2.0,0.0,home_win
44102,0.0,0.0,draw


In [48]:
y_es = df_label.drop('win', axis=1).to_numpy()
x_es = df_temp.drop('city', axis=1).to_numpy()
x_t, x_val , y_t, y_val = train_test_split(x_es, y_es, random_state=28)

In [49]:
#x_t=pd.DataFrame(data=x_temp)
#x_t

In [50]:
params = {
    "estimator__colsample_bylevel": uniform(0.7, 0.3),
    "estimator__gamma": uniform(0, 0.5),
    "estimator__learning_rate": uniform(0.003, 0.3),
    "estimator__max_depth": randint(2, 6),
    "estimator__n_estimators": randint(100, 500), 
    "estimator__subsample": uniform(0.6, 0.4)
}

In [51]:
#Multioutput regression are regression problems that involve predicting two or more numerical values given an input example.
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', eval_metric=['mae'], random_state = 1, use_label_encoder=False))

In [52]:
#search : 하이퍼 파라미터 튜닝
search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state =1, n_iter=5, cv=3,
                            verbose=0, n_jobs=1, return_train_score=True) 
search.fit(x_t, y_t) 

best_param = search.best_params_
print('-')
print(search.best_score_)
print(best_param)
print('-')

-
0.10852745493991223
{'estimator__colsample_bylevel': 0.7277015784306393, 'estimator__gamma': 0.09313010568883545, 'estimator__learning_rate': 0.10666821811291433, 'estimator__max_depth': 4, 'estimator__n_estimators': 381, 'estimator__subsample': 0.8155266936013428}
-


In [53]:
#RandomizedSearchCV로 찾은 best param을 모델에 적용
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror',eval_metric=['mae'], random_state = 1, 
                              use_label_encoder=False, **best_param)).fit(x_t, y_t) 

In [54]:
#MAE : MSE와 유사한 loss function
#이 코드에서 loss function이 적용되는 부분이 어디지? 못 찾았음
print('MAE: ',mean_absolute_error(y_val, xgb_model.predict(x_val)))

MAE:  1.0839428344574897


In [55]:
#K-fold 교차 검증:
#전체 훈련 데이터를 K개의 그룹으로 묶고, 그중 한 그룹을 검증셋으로 사용합니다. 
#이때 각각의 모든 데이터 그룹이 적어도 한 번씩은 검증셋으로 사용되어야 합니다. 
#K-fold 교차 검증을 사용함으로써 검정 데이터에 모델이 과적합 되는 것을 막을 수 있습니다. 
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(xgb_model, x_val, y_val, scoring='neg_mean_absolute_error', cv=folds, verbose=0)
print('KFold_Neg_MAE: ',np.mean(scores))

KFold_Neg_MAE:  -1.0911797128265908


TEST 32

In [56]:
#inp_list: 사용자가 입력한 home_team, away team
def predict_score(inp_list, autofill, Norm, seed, model):
    ind_list = []
    for i in inp_list:
        ind_list.append(nat_dic.get(i))
    #seed의 목적이 뭐지?
    seed[:,1] = ind_list[0]
    seed[:,2] = ind_list[1]
    x_test = Norm.fit_transform(seed)
    pred = model.predict(x_test)
    return np.ravel(pred).tolist()

In [57]:
#예선 팀 조 편성 결과 입력
Group_A= ["Uruguay","Russia","Saudi Arabia","Egypt"]
Group_B= ["Portugal","Spain","Morocco","Iran"]
Group_C= ["France","Denmark","Australia","Peru"]
Group_D= ["Argentina","Croatia","Iceland","Nigeria"]
Group_E= ["Brazil","Switzerland","Costa Rica","Serbia"]
Group_F= ["South Korea","Mexico","Sweden","Germany"]
Group_G= ["Belgium","England","Panama","Tunisia"]
Group_H= ["Senegal","Japan","Poland","Colombia"]
Groups={"Group A":Group_A,"Group B":Group_B,"Group C":Group_C,"Group D":Group_D,"Group E":Group_E,"Group F":Group_F,"Group G":Group_G,"Group H":Group_H}

In [58]:
"""
#all_countries
df_countries = pd.read_csv('./results.csv')
countries_home= (df_countries['home_team'].unique()).tolist()
countries_away = (df_countries['away_team'].unique()).tolist()
all_countries = countries_home + countries_away
all_countires = list(set(all_countries))
"""

"\n#all_countries\ndf_countries = pd.read_csv('./results.csv')\ncountries_home= (df_countries['home_team'].unique()).tolist()\ncountries_away = (df_countries['away_team'].unique()).tolist()\nall_countries = countries_home + countries_away\nall_countires = list(set(all_countries))\n"

In [59]:
"""
#label_ecoder의 역할?
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
#Labeling the data using LabelEncorder in Sklearn-(Giving a unique number to each string(country))
label_encoder.fit(all_countries)
"""

'\n#label_ecoder의 역할?\nfrom sklearn import preprocessing\nlabel_encoder = preprocessing.LabelEncoder()\n#Labeling the data using LabelEncorder in Sklearn-(Giving a unique number to each string(country))\nlabel_encoder.fit(all_countries)\n'

In [60]:
def create_input(home_team, away_team):
  #인풋으로 home_team, away_team만 주면 되는가? 아니면 별도의 벡터 생성 필요?
  ind_list = [home_team, away_team]
  return ind_list#input_vector

In [61]:
def create_params():
  seed = x_train[random.randint(0,x_train.shape[0])].reshape(1,-1)
  seed = np.delete(seed, 4, axis=1)
  params = {
    'autofill': True,
    'Norm': Norm,
    'seed': seed,
    'model': xgb_model
  }
  return params

In [62]:
def select_winning_team(prob_lst):
    prob_lst[0] = round(prob_lst[0])
    prob_lst[1] = round(prob_lst[1])
    #prob_lst=[round(probability_array[0][i],3) for i in range(2)]
    if (prob_lst[0]>prob_lst[1]):
        out=0
    elif (prob_lst[0]<prob_lst[1]):
        out=1
    elif (prob_lst[0]==prob_lst[1]):
        out=2
    return out,prob_lst

In [63]:
##Group stage Matches
Group_standings={}
for grp_name in list(Groups.keys()):
    print(f"{grp_name} Matches")
    #해당 그룹의 팀들
    probable_countries=Groups[grp_name]

    team_wins_dct={}
    goal_scored_dct={}
    goal_against_dct={}
    win_dct={}
    draw_dct={}
    lost_dct={}
    
    for i in range(len(probable_countries)):
        #team i: team1, team j : team2
        j=i+1
        team_1=probable_countries[i]
        #team_1_num=label_encoder.transform([team_1])[0]

        team_wins=0
    
        while j<len((probable_countries)):
            team_2=probable_countries[j]
            #team_2_num=label_encoder.transform([team_2])[0]

            team_lst=[team_1,team_2]
            #print("\n",team_lst)
            #input 생성
            #인풋으로 home_team, away_team만 주면 되는가? 아니면 별도의 벡터 생성 필요?
            input=create_input(team_1, team_2)#np.array([[year,stadium_num,team_1_num,team_2_num]])
            params = create_params()
            #res가 왜 소수점으로 나올까?
            #임시방편으로 반올림
            res=predict_score(input, **params)#model.predict(Input_vector)
            
            #res[0]=round(res[0])
            #res[1]=round(res[1])
            #print(res[0], res[1])

            #select_winning team 파라미터 변경
            win,prob_lst=select_winning_team(res)
            goal_scored_dct[team_1] = goal_scored_dct.get(team_1,0)+prob_lst[0]
            goal_scored_dct[team_2] = goal_scored_dct.get(team_2,0)+prob_lst[1]

            goal_against_dct[team_1] = goal_against_dct.get(team_1,0)+prob_lst[1]
            goal_against_dct[team_2] = goal_against_dct.get(team_2,0)+prob_lst[0]
            
            try:
                print(f" {team_1} vs {team_2} \n  Results of the Match {res[0]}\n   {team_lst[win]} wins 🎊🏆⚽🎖️\n")
                if (win)==0:
                    team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+2
                    team_wins_dct[team_2] = team_wins_dct.get(team_2,0)
                    
                    win_dct[team_1] = win_dct.get(team_1,0)+1
                    win_dct[team_2] = win_dct.get(team_2,0)
                    lost_dct[team_2] = lost_dct.get(team_2,0)+1
                    lost_dct[team_1] = lost_dct.get(team_1,0)
                    draw_dct[team_2] = draw_dct.get(team_2,0)
                    draw_dct[team_1] = draw_dct.get(team_1,0)

                elif (win)==1:
                    team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+2
                    team_wins_dct[team_1] = team_wins_dct.get(team_1,0)
                    
                    win_dct[team_2] = win_dct.get(team_2,0)+1
                    win_dct[team_1] = win_dct.get(team_1,0)
                    lost_dct[team_1] = lost_dct.get(team_1,0)+1
                    lost_dct[team_2] = lost_dct.get(team_2,0)
                    draw_dct[team_1] = draw_dct.get(team_1,0)
                    draw_dct[team_2] = draw_dct.get(team_2,0)

            except IndexError:
                print(f"{team_1} vs {team_2} \n  Results of the Match {res[0]}\n   Match Draw ⚽⚽⚽\n") 
                team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+1
                team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+1
                
                draw_dct[team_1] = draw_dct.get(team_1,0)+1
                draw_dct[team_2] = draw_dct.get(team_2,0)+1
                
                win_dct[team_1] = win_dct.get(team_1,0)
                lost_dct[team_1] = lost_dct.get(team_1,0)
                
                win_dct[team_2] = win_dct.get(team_2,0)
                lost_dct[team_2] = lost_dct.get(team_2,0)

            j=j+1
            
    group_results=[win_dct,draw_dct,lost_dct,team_wins_dct,goal_scored_dct,goal_against_dct]
    Group_standings[grp_name]=group_results

Group A Matches
Uruguay vs Russia 
  Results of the Match 2
   Match Draw ⚽⚽⚽

 Uruguay vs Saudi Arabia 
  Results of the Match 4
   Uruguay wins 🎊🏆⚽🎖️

 Uruguay vs Egypt 
  Results of the Match 3
   Uruguay wins 🎊🏆⚽🎖️

 Russia vs Saudi Arabia 
  Results of the Match 3
   Russia wins 🎊🏆⚽🎖️

 Russia vs Egypt 
  Results of the Match 3
   Russia wins 🎊🏆⚽🎖️

 Saudi Arabia vs Egypt 
  Results of the Match 4
   Saudi Arabia wins 🎊🏆⚽🎖️

Group B Matches
Portugal vs Spain 
  Results of the Match 1
   Match Draw ⚽⚽⚽

 Portugal vs Morocco 
  Results of the Match 1
   Morocco wins 🎊🏆⚽🎖️

 Portugal vs Iran 
  Results of the Match 3
   Portugal wins 🎊🏆⚽🎖️

 Spain vs Morocco 
  Results of the Match 1
   Morocco wins 🎊🏆⚽🎖️

 Spain vs Iran 
  Results of the Match 3
   Spain wins 🎊🏆⚽🎖️

 Morocco vs Iran 
  Results of the Match 4
   Morocco wins 🎊🏆⚽🎖️

Group C Matches
France vs Denmark 
  Results of the Match 2
   Match Draw ⚽⚽⚽

 France vs Australia 
  Results of the Match 2
   France wins 🎊🏆⚽🎖️

France

In [64]:
"""
##Group stage Matches
Group_standings={}
for grp_name in list(Groups.keys()):
    print(f"{grp_name} Matches")
    print(Groups[grp_name])
    probable_countries=Groups[grp_name]
    team_wins_dct={}
    goal_scored_dct={}
    goal_against_dct={}
    win_dct={}
    draw_dct={}
    lost_dct={}
    
    for i in range(len(probable_countries)):
        j=i+1
        team_1=probable_countries[i]
        team_1_num=label_encoder.transform([team_1])[0]

        team_wins=0
    
        while j<len((probable_countries)):
            team_2=probable_countries[j]
            team_2_num=label_encoder.transform([team_2])[0]

            team_lst=[team_1,team_2]

            #input vector 생성
            Input_vector=create_input_vector(team_1, team_2)#np.array([[year,stadium_num,team_1_num,team_2_num]])
            params = create_params()
            res=predict_score(Input_vector, **params)#model.predict(Input_vector)

          
            win,prob_lst=select_winning_team(res)
            goal_scored_dct[team_1] = goal_scored_dct.get(team_1,0)+prob_lst[0]
            goal_scored_dct[team_2] = goal_scored_dct.get(team_2,0)+prob_lst[1]

            goal_against_dct[team_1] = goal_against_dct.get(team_1,0)+prob_lst[1]
            goal_against_dct[team_2] = goal_against_dct.get(team_2,0)+prob_lst[0]

            try:
                print(f" {team_1} vs {team_2} \n  Results of the Match {res[0]}\n   {team_lst[win]} wins 🎊🏆⚽🎖️\n")
                if (win)==0:
                    team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+2
                    team_wins_dct[team_2] = team_wins_dct.get(team_2,0)
                    
                    win_dct[team_1] = win_dct.get(team_1,0)+1
                    win_dct[team_2] = win_dct.get(team_2,0)
                    lost_dct[team_2] = lost_dct.get(team_2,0)+1
                    lost_dct[team_1] = lost_dct.get(team_1,0)
                    draw_dct[team_2] = draw_dct.get(team_2,0)
                    draw_dct[team_1] = draw_dct.get(team_1,0)

                elif (win)==1:
                    team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+2
                    team_wins_dct[team_1] = team_wins_dct.get(team_1,0)
                    
                    win_dct[team_2] = win_dct.get(team_2,0)+1
                    win_dct[team_1] = win_dct.get(team_1,0)
                    lost_dct[team_1] = lost_dct.get(team_1,0)+1
                    lost_dct[team_2] = lost_dct.get(team_2,0)
                    draw_dct[team_1] = draw_dct.get(team_1,0)
                    draw_dct[team_2] = draw_dct.get(team_2,0)

            except IndexError:
                print(f"{team_1} vs {team_2} \n  Results of the Match {res[0]}\n   Match Draw ⚽⚽⚽\n") 
                team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+1
                team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+1
                
                draw_dct[team_1] = draw_dct.get(team_1,0)+1
                draw_dct[team_2] = draw_dct.get(team_2,0)+1
                
                win_dct[team_1] = win_dct.get(team_1,0)
                lost_dct[team_1] = lost_dct.get(team_1,0)
                
                win_dct[team_2] = win_dct.get(team_2,0)
                lost_dct[team_2] = lost_dct.get(team_2,0)
                    
            j=j+1
    group_results=[win_dct,draw_dct,lost_dct,team_wins_dct,goal_scored_dct,goal_against_dct]
    Group_standings[grp_name]=group_results
""" 

'\n##Group stage Matches\nGroup_standings={}\nfor grp_name in list(Groups.keys()):\n    print(f"{grp_name} Matches")\n    print(Groups[grp_name])\n    probable_countries=Groups[grp_name]\n    team_wins_dct={}\n    goal_scored_dct={}\n    goal_against_dct={}\n    win_dct={}\n    draw_dct={}\n    lost_dct={}\n    \n    for i in range(len(probable_countries)):\n        j=i+1\n        team_1=probable_countries[i]\n        team_1_num=label_encoder.transform([team_1])[0]\n\n        team_wins=0\n    \n        while j<len((probable_countries)):\n            team_2=probable_countries[j]\n            team_2_num=label_encoder.transform([team_2])[0]\n\n            team_lst=[team_1,team_2]\n\n            #input vector 생성\n            Input_vector=create_input_vector(team_1, team_2)#np.array([[year,stadium_num,team_1_num,team_2_num]])\n            params = create_params()\n            res=predict_score(Input_vector, **params)#model.predict(Input_vector)\n\n          \n            win,prob_lst=select

In [65]:
"""역할?

#list(label_encoder.classes_)
categorized_data['team_1']=label_encoder.transform(categorized_data['team_1'])
categorized_data['team_2']=label_encoder.transform(categorized_data['team_2'])
categorized_data['Country']=label_encoder.transform(categorized_data['Country'])

#Converting these feature columns to categrize form to make the training processs more smoother
categorized_data['team_1']=categorized_data['team_1'].astype("category")
categorized_data['team_2']=categorized_data['team_2'].astype("category")
categorized_data['Country']=categorized_data['team_2'].astype("category")
"""

'역할?\n\n#list(label_encoder.classes_)\ncategorized_data[\'team_1\']=label_encoder.transform(categorized_data[\'team_1\'])\ncategorized_data[\'team_2\']=label_encoder.transform(categorized_data[\'team_2\'])\ncategorized_data[\'Country\']=label_encoder.transform(categorized_data[\'Country\'])\n\n#Converting these feature columns to categrize form to make the training processs more smoother\ncategorized_data[\'team_1\']=categorized_data[\'team_1\'].astype("category")\ncategorized_data[\'team_2\']=categorized_data[\'team_2\'].astype("category")\ncategorized_data[\'Country\']=categorized_data[\'team_2\'].astype("category")\n'

In [66]:
#label encoder의 역할?
year=2022
stadium="Qatar"
stadium_num=label_encoder.transform([stadium])[0]
host_num=stadium_num